In [2]:
import random
from collections import defaultdict, deque

def generate_large_dataset(num_classes, num_rooms, num_teachers, num_times):
    classes = {f'Class_{i}': f'Teacher_{random.randint(1, num_teachers)}' for i in range(num_classes)}
    rooms = [f'Room_{i}' for i in range(1, num_rooms + 1)]
    teachers = [f'Teacher_{i}' for i in range(1, num_teachers + 1)]
    times = [f'{i}pm' for i in range(1, 1 + num_times)]

    room_availability = {room: random.sample(times, k=random.randint(1, num_times)) for room in rooms}
    teacher_availability = {teacher: random.sample(times, k=random.randint(1, num_times)) for teacher in teachers}

    preassigned_classes = {}
    for cls in classes:
        if random.choice([True, False]):
            room = random.choice(rooms)
            time = random.choice(room_availability[room])
            preassigned_classes[cls] = (room, time)

    return classes, rooms, teachers, times, room_availability, teacher_availability, preassigned_classes


In [3]:

class ClassScheduleCSP:
    def __init__(self, classes, rooms, teachers, times, room_availability, teacher_availability, preassigned_classes):
        self.classes = classes
        self.rooms = rooms
        self.teachers = teachers
        self.times = times
        self.room_availability = room_availability
        self.teacher_availability = teacher_availability
        self.preassigned_classes = preassigned_classes
        self.domains = self.initialize_domains()
        self.constraints = self.initialize_constraints()

    def initialize_domains(self):
        domains = {}
        for cls in self.classes:
            if cls in self.preassigned_classes:
                domains[cls] = [self.preassigned_classes[cls]]
            else:
                domains[cls] = [(room, time) for room in self.rooms for time in self.times
                                if room in self.room_availability and time in self.room_availability[room]
                                and time in self.teacher_availability[self.classes[cls]]]
        return domains

    def initialize_constraints(self):
        constraints = defaultdict(list)
        for cls1 in self.classes:
            for cls2 in self.classes:
                if cls1 != cls2:
                    constraints[cls1].append((cls2, self.class_constraint))
        return constraints

    def class_constraint(self, cls1, assignment1, cls2, assignment2):
        room1, time1 = assignment1
        room2, time2 = assignment2
        teacher1 = self.classes[cls1]
        teacher2 = self.classes[cls2]
        return (room1 != room2 or time1 != time2) and (teacher1 != teacher2 or time1 != time2)

    def is_consistent(self, cls, value, assignment):
        for neighbor in self.constraints[cls]:
            neighbor_cls, constraint = neighbor
            if neighbor_cls in assignment and not constraint(cls, value, neighbor_cls, assignment[neighbor_cls]):
                return False
        return True

    def select_unassigned_variable(self, assignment):
        unassigned = [cls for cls in self.classes if cls not in assignment]

        def degree_heuristic(var):
            return sum(1 for neighbor in self.constraints[var] if neighbor[0] not in assignment)

        return min(unassigned, key=lambda var: (len(self.domains[var]), -degree_heuristic(var)))

    def order_domain_values(self, var, assignment):
        return sorted(self.domains[var], key=lambda val: self.lcv_heuristic(var, val, assignment))

    def lcv_heuristic(self, var, value, assignment):
        count = 0
        for neighbor in self.constraints[var]:
            neighbor_cls, _ = neighbor
            if neighbor_cls not in assignment:
                for neighbor_value in self.domains[neighbor_cls]:
                    if not self.class_constraint(var, value, neighbor_cls, neighbor_value):
                        count += 1
        return count

    def ac3(self):
        queue = deque([(var, neighbor[0]) for var in self.constraints for neighbor in self.constraints[var]])
        while queue:
            (var1, var2) = queue.popleft()
            if self.revise(var1, var2):
                if not self.domains[var1]:
                    return False
                for neighbor in self.constraints[var1]:
                    if neighbor[0] != var2:
                        queue.append((neighbor[0], var1))
        return True

    def revise(self, var1, var2):
        revised = False
        for value in self.domains[var1][:]:
            if not any(self.class_constraint(var1, value, var2, value2) for value2 in self.domains[var2]):
                self.domains[var1].remove(value)
                revised = True
        return revised

    def backtracking_search(self):
        self.ac3()
        return self.backtrack({})

    def forward_check(self, var, value, assignment):
        for neighbor in self.constraints[var]:
            neighbor_cls, _ = neighbor
            if neighbor_cls not in assignment:
                if not any(self.class_constraint(var, value, neighbor_cls, neighbor_value)
                           for neighbor_value in self.domains[neighbor_cls]):
                    return False
        return True

    def backtrack(self, assignment):
        if len(assignment) == len(self.classes):
            return assignment

        var = self.select_unassigned_variable(assignment)
        for value in self.order_domain_values(var, assignment):
            if self.is_consistent(var, value, assignment):
                assignment[var] = value
                if self.forward_check(var, value, assignment):
                    result = self.backtrack(assignment)
                    if result:
                        return result
                assignment.pop(var)
        return None


In [4]:

def find_valid_schedule():
    while True:
        classes, rooms, teachers, times, room_availability, teacher_availability, preassigned_classes = generate_large_dataset(10, 5, 3, 5)
        csp = ClassScheduleCSP(classes, rooms, teachers, times, room_availability, teacher_availability, preassigned_classes)
        solution = csp.backtracking_search()

        if solution:
            print("Solution found:")
            for cls, (room, time) in solution.items():
                teacher = classes[cls]
                print(f"Class {cls} with {teacher} in {room} at {time}")
            break
        else:
            print("No solution found, generating a new dataset and trying again...")

# Run the main loop
find_valid_schedule()


No solution found, generating a new dataset and trying again...
No solution found, generating a new dataset and trying again...
No solution found, generating a new dataset and trying again...
No solution found, generating a new dataset and trying again...
No solution found, generating a new dataset and trying again...
No solution found, generating a new dataset and trying again...
Solution found:
Class Class_3 with Teacher_3 in Room_3 at 5pm
Class Class_5 with Teacher_3 in Room_2 at 3pm
Class Class_8 with Teacher_3 in Room_3 at 2pm
Class Class_2 with Teacher_3 in Room_1 at 1pm
Class Class_4 with Teacher_3 in Room_1 at 4pm
Class Class_0 with Teacher_1 in Room_1 at 3pm
Class Class_1 with Teacher_1 in Room_1 at 2pm
Class Class_6 with Teacher_2 in Room_2 at 1pm
Class Class_7 with Teacher_2 in Room_4 at 3pm
Class Class_9 with Teacher_2 in Room_1 at 5pm
